In [48]:
import os

os.makedirs("/content/tacotron2", exist_ok=True)
os.makedirs("/content/waveglow", exist_ok=True)
os.makedirs("/content/checkpoints", exist_ok=True)


In [22]:
!pip install -q torch torchvision torchaudio
!pip install -q librosa matplotlib unidecode inflect tensorboard


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.1 MB/s eta 0:00:00


In [49]:
!mkdir -p /content/checkpoints

In [50]:
import torch
import numpy as np
import torchaudio
from scipy.io.wavfile import write


In [72]:
!git clone https://github.com/NVIDIA/tacotron2.git



fatal: destination path 'tacotron2' already exists and is not an empty directory.


In [51]:
!pip install -q torch torchvision torchaudio
!pip install -q unidecode librosa matplotlib scipy numpy inflect

In [81]:
from google.colab import files
uploaded = files.upload()


Saving nvidia_tacotron2pyt_fp32_20190427 to nvidia_tacotron2pyt_fp32_20190427


In [84]:
import os

# List files in /content to find the correct WaveGlow filename
for f in os.listdir("/content"):
    print(f)


.config
data
output.wav
checkpoints
waveglow_256channels_universal_v5.pt
waveglow
preprocess.py
__pycache__
hifi-gan
tacotron
TensorFlowTTS
LJSpeech-1.1.tar.bz2
DeepLearningExamples
tacotron2
sample_data


In [72]:
from google.colab import files
uploaded = files.upload()



Saving waveglow_256channels_ljs_v3.pt to waveglow_256channels_ljs_v3.pt


In [85]:
import os

# List everything in /content
for f in os.listdir("/content"):
    print(f)


.config
data
output.wav
checkpoints
waveglow_256channels_universal_v5.pt
waveglow
preprocess.py
__pycache__
hifi-gan
tacotron
TensorFlowTTS
LJSpeech-1.1.tar.bz2
DeepLearningExamples
tacotron2
sample_data


In [86]:
import os

# Walk through all directories and look for the checkpoint
for root, dirs, files in os.walk("/content"):
    for file in files:
        if "waveglow" in file and file.endswith(".pt"):
            print(os.path.join(root, file))


/content/waveglow_256channels_universal_v5.pt
/content/tacotron/waveglow/waveglow_256channels_universal_v5.pt


In [54]:
import tensorflow as tf


In [88]:
from argparse import Namespace


In [88]:
!cat /content/tacotron2/hparams.py


from argparse import Namespace

def create_hparams(hparams_string=None, verbose=False):
    """Create model hyperparameters."""

    hparams = Namespace(
        ################################
        # Experiment Parameters        #
        ################################
        epochs=500,
        iters_per_checkpoint=1000,
        seed=1234,
        dynamic_loss_scaling=True,
        fp16_run=False,
        distributed_run=False,
        dist_backend="nccl",
        dist_url="tcp://localhost:54321",
        cudnn_enabled=True,
        cudnn_benchmark=False,
        ignore_layers=['embedding.weight'],

        ################################
        # Data Parameters              #
        ################################
        load_mel_from_disk=False,
        training_files='filelists/ljs_audio_text_train_filelist.txt',
        validation_files='filelists/ljs_audio_text_val_filelist.txt',
        text_cleaners=['english_cleaners'],

        ################################
 

In [91]:
# Step 1: Remove hparams from cache
import sys
if 'hparams' in sys.modules:
    del sys.modules['hparams']

# Step 2: Overwrite hparams.py manually again
hparams_code = '''from argparse import Namespace

def create_hparams(hparams_string=None, verbose=False):
    """Create model hyperparameters."""

    hparams = Namespace(
        ################################
        # Experiment Parameters        #
        ################################
        epochs=500,
        iters_per_checkpoint=1000,
        seed=1234,
        dynamic_loss_scaling=True,
        fp16_run=False,
        distributed_run=False,
        dist_backend="nccl",
        dist_url="tcp://localhost:54321",
        cudnn_enabled=True,
        cudnn_benchmark=False,
        ignore_layers=['embedding.weight'],

        ################################
        # Data Parameters              #
        ################################
        load_mel_from_disk=False,
        training_files='filelists/ljs_audio_text_train_filelist.txt',
        validation_files='filelists/ljs_audio_text_val_filelist.txt',
        text_cleaners=['english_cleaners'],

        ################################
        # Audio Parameters             #
        ################################
        max_wav_value=32768.0,
        sampling_rate=22050,
        filter_length=1024,
        hop_length=256,
        win_length=1024,
        n_mel_channels=80,
        mel_fmin=0.0,
        mel_fmax=8000.0,

        ################################
        # Model Parameters             #
        ################################
        n_symbols=148,
        symbols_embedding_dim=512,

        encoder_kernel_size=5,
        encoder_n_convolutions=3,
        encoder_embedding_dim=512,

        n_frames_per_step=1,
        decoder_rnn_dim=1024,
        prenet_dim=256,
        max_decoder_steps=1000,
        gate_threshold=0.5,
        p_attention_dropout=0.1,
        p_decoder_dropout=0.1,

        attention_rnn_dim=1024,
        attention_dim=128,

        attention_location_n_filters=32,
        attention_location_kernel_size=31,

        postnet_embedding_dim=512,
        postnet_kernel_size=5,
        postnet_n_convolutions=5,

        ################################
        # Optimization Hyperparameters #
        ################################
        use_saved_learning_rate=False,
        learning_rate=1e-3,
        weight_decay=1e-6,
        grad_clip_thresh=1.0,
        batch_size=32,
        mask_padding=True
    )

    return hparams
'''

with open("/content/tacotron2/hparams.py", "w") as f:
    f.write(hparams_code)

print("✅ Clean hparams.py written and cache cleared.")


✅ Clean hparams.py written and cache cleared.


In [156]:
!pip install numpy scipy matplotlib librosa unidecode


In [60]:
!git clone https://github.com/jik876/hifi-gan.git

fatal: destination path 'hifi-gan' already exists and is not an empty directory.


In [61]:
!pip install -q numpy scipy matplotlib ipython librosa unidecode inflect tqdm
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.5 MB/s eta 0:00:00


In [62]:
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
%cd TensorFlowTTS
!pip install -e .

# Other required libs
!pip install soundfile ipywidgets

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 10679, done.
remote: Total 10679 (delta 0), reused 0 (delta 0), pack-reused 10679 (from 1)
Receiving objects: 100% (10679/10679), 133.52 MiB | 12.42 MiB/s, done.
Resolving deltas: 100% (5163/5163), done.
/content/TensorFlowTTS
Obtaining file:///content/TensorFlowTTS
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 72.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Gettin

In [202]:
!pip install numpy torch gdown matplotlib Unidecode


  Using cached Unidecode-1.3.8-py3-none-any.whl.metadata (13 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 137.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.8 MB/s eta 0:00:00
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [63]:
# STEP 1: Load Tacotron2 and HiFi-GAN from TorchHub
import torch

# Tacotron2 (Text → Mel)
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2')
tacotron2 = tacotron2.eval().cuda()

# HiFi-GAN (Mel → Audio)
hifigan = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_hifigan')[0]
hifigan = hifigan.eval().cuda()


Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub
Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


HiFi-GAN: Removing weight norm.


In [94]:
# User input
user_text = input("Enter the text you want to convert to speech: ")

# Process the text
from tacotron2.text import text_to_sequence
import torch
sequence = text_to_sequence(user_text, ['english_cleaners'])
sequence = torch.IntTensor(sequence)[None, :].to('cuda')
input_lengths = torch.LongTensor([sequence.shape[1]]).to('cuda')

# Generate mel
with torch.no_grad():
    mel_outputs, mel_lengths, alignments = tacotron2.infer(sequence, input_lengths)

# Convert mel to audio
mel = mel_outputs.to('cuda')
with torch.no_grad():
    audio = hifigan(mel)

# Convert audio tensor to numpy
audio_np = audio.squeeze().cpu().numpy()

# Play the audio in output
from IPython.display import Audio, display
display(Audio(audio_np, rate=22050))  # Display is important here to play it in Colab

# Save and download the audio
from scipy.io.wavfile import write
from google.colab import files

write('output.wav', 22050, audio_np)     # Save as WAV
files.download('output.wav')             # Trigger download


Enter the text you want to convert to speech: This is my coab notebook for AI powered speech synthesis


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>